In [71]:
import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
import data_utils as du 
from cnn_model import *
from solver import Solver
import qunatize_resnet20 as q_res
import quantize_layers as ql
import tensorflow_hub as hub 
import quantize_util as qu 
import quantize_complier as qc 


%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload
plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'
X_train, y_train, X_val, y_val, X_test, y_test = du.get_CIFAR10_data(subtract_mean =True)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [72]:
data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 1,
    'from_logits':True,
    'save_dir': 'resnet20_model'
}
model_path = 'resnet20_model/trained_15'
full_model = hub.KerasLayer(model_path,trainable=True)

qunatizer = ql.ModelQunatize() 
model_q = q_res.Resnet20(qunatizer)
qu.copy_weight(model_q,full_model)
q_layers = model_q.get_quantizable_layers()
for layer in q_layers:
    print(layer)
solver = Solver(model_q,data_set,train_options,save_model=False)

def train_fn(model):
    solver = Solver(model,data_set,train_options,
                    save_model=False,verbose=False,
                    plot_graph=False)
    return solver.train(iter=int(5000/32))

def test_fn(model):
    solver = Solver(model,data_set,save_model=False,
                    verbose=False,plot_graph=False)
    return solver.test_model(X_test,y_test)





In [73]:
q_complier = qc.QunatizeComplier(model_q,full_model,q_layers,train_fn,test_fn,verbose=True,target_compression=1.2,max_loss=0.005)
q_complier.quantize_model()

wl on layer 10 kernel,acc_loss: 0.0, current accuracy:0.6870999932289124, compression rate: (0.208832999187616, 1793264)
<quantize_complier.WlflList object at 0x7f6ecddb0ca0>[[5,5,0,0][5,5,0,0][5,5,0,0][4,4,0,0][3,4,0,0][3,4,0,0][4,4,0,0][6,6,0,0][6,6,0,0][5,5,0,0][4,5,0,0][7,7,0,0][5,5,0,0][7,7,0,0][7,7,0,0][7,7,0,0][7,7,0,0][7,7,0,0][8,8,0,0][5,4,0,0]]
<quantize_complier.AccuracyLossPerBitList object at 0x7f6eb0372490>[[1.2731524293485563e-05,None][5.121523827256169e-06,None][3.2986151836666977e-06,None][2.7777730338129913e-06,None][2.951387159555452e-06,None][4.991319656255655e-06,None][1.0416842997074127e-06,None][7.378532700386131e-07,None][2.3871609755587997e-07,None][3.146706148982048e-07,None][-1.4105656021001778e-07,None][2.0616506901660614e-07,None][3.6892663501930656e-07,None][5.425600591024704e-08,None][4.3403513672046756e-08,None][0.0,None][-2.9839512194484996e-08,None][-2.1700138574942684e-08,None][8.137754470283198e-09,None][2.2343732780427672e-05,1]]
reduce wl on layer 

(<qunatize_resnet20.Resnet20 at 0x7f6ece3f22b0>,
 <quantize_complier.WlflList at 0x7f6ecddb0ca0>)

In [74]:

model_save_path = 'complier_tmp/max_loss_0005_deep_compressed_resnet20_model'
wlfl_save_path = 'complier_tmp/max_loss_0005_deep_compressed_resnet20_wlfl.txt'
model_q.model.save_weights(model_save_path)
tf.saved_model.save(model_q.model,model_save_path)
q_complier.wl_agent.get_wl_list().save(wlfl_save_path)


INFO:tensorflow:Assets written to: complier_tmp/max_loss_0005_deep_compressed_resnet20_model/assets


In [75]:

model_q.model.load_weights(model_save_path)
q_layers = model_q.get_quantizable_layers()
wlfl_list = qc.WlflList(len(q_layers))
wlfl_list.load(wlfl_save_path)
q_complier.fl_agent.apply_wlfl_to_layers(q_layers,wlfl_list)
solver = Solver(model_q,data_set,train_options)
solver.test_model(X_test,y_test)
q_complier.wl_agent.get_compression_rate()


Loss: 1.4968599081039429,Test Accuracy: 68.5199966430664


(0.15570755666192154, 1337072)

In [76]:
solver.test_model(X_train,y_train)

Loss: 0.788412868976593,Test Accuracy: 79.06735229492188


(<tf.Tensor: shape=(), dtype=float32, numpy=0.78841287>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.7906735>)